# **PRÁCTICA 7:  Machine Learning - Proyecto final**
### Universitat de València, Escola Tecnica Superior d'Enginyeria
### Elena Marrero Castellano | 3ª curso del Grado Ciencia de Datos

Desarrolla un algoritmo que a partir de unos datos de entrada (X) prediga los valores de salida (Y). La nota no va a depender únicamente de los resultados obtenidos sino de el procedimiento utilizado. Esmérate en incluir comentarios que describan el procedimiento y la motivación de cada paso. Se valorará el uso de todos los conocimientos desarrollados durante las prácticas. Entre otros:

- Métodos de preprocesado.
- Métodos de selección de características.
- Métodos de extracción de características.
- Métodos de partición de datos durante el entrenamiento.
- Métodos de clasificación/regresión, cuantos más se prueben mejor.

Entregaréis:
- Un fichero 'y_hat.cvs' con vuestra predicciones sobre los datos de test.
- Un fichero '.ipynb' que contenga el procedimiento utilizado debidamente comentado.

## Comenzamos

Nuestro objetivo va a ser entrenar un modelo para ver si luego funciona correctamente. Antes de empezar tenemos que descargar las librerias necesarias para la realización de la práctica

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist
from scipy.stats import mode
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, QuantileTransformer, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import sklearn.metrics

## Adquisición de los datos

Empezamos como siempre descargando nuestros datos y a partir de los mismos dividimos el conjunto inicial de los datos en dos. Normalmente deberemos coger 2/3 del conjunto de datos inicial para entrenar (siendo Xtrain, ytrain los nombres de dichas variables), por otro lado el 1/3 restante para probar los datos y ver si funcionan correctamete (siendo Xtest, ytest los nombres de dichas variables). Con las variables Xtest, ytest no se trabaja con ellas en entrenamiento, asegurandonos así que no conozcan los datos reales para poderlos tener en cuenta a la hora de definir como de bueno es el modelo. Pero en este caso no vamos a dicidir el conjunto de datos de esta forma ya que tenemos ya la adquisición de datos dada.

In [ ]:
# Descargamos 
# Read data and start to work
df_train = pd.read_table('diabetes_train.data')
df_test = pd.read_table('diabetes_test.data')

# Nombres de las variables
print(df_train.keys())
data_train = df_train.to_numpy()

ytrain, ytrain_class = data_train[:,-2], data_train[:,-1]
Xtrain = data_train[:, :-2]

# Test set
Xtest = df_test.to_numpy()

Index(['AGE', 'SEX', 'BMI', 'BP', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'Y',
       'C'],
      dtype='object')


## Preprocesado de los datos

Comenzamos realizando una normalización de nuestros datos porque mchos modelos pueden sensibles a los valores de las características implicadas o para entender esa relación entre las variables. Todas las muestras deben normalizarse con los parámetros correspondientes al conjunto de entrenamiento. Nosotros en este curso hemos visto varias de ellas, la normalizacion, la normalización quitando el mínimo, la estandarización o usando los modelos de normalización (MinMaxScaler, MaxAbsScaler o StandardScaler). En este caso voy a realizarlo con el MinMaxScaler ya tiene un parámetro que nos dice el rango el cual queremos meter los datos (feature_range), por defecto sería entre 0 y 1. Este método, como sabemos se utiliza el escalado entre los rangos que yo los busco.

In [ ]:
# Normalizamos
scaler = MinMaxScaler()

# Ajuste solo en el set de entrenamiento.
scaler.fit(Xtrain)

# Aplicamos la transformación tanto al conjunto de train como al conjunto de test.
X_train_scaled = scaler.transform(Xtrain)
X_test_scaled = scaler.transform(Xtest)

# Podemos transformar estos datos a escalado o trabajar directamente con ellos, vamos a probar las dos formas 
# y_train_scaled = scaler2.transform(ytrain)
y_train = ytrain

Además de la normalización, en ocasiones se requiere una etapa adicional de codificación porque los modelos pueden necesitar que las variables presenten un formato específico: (numérico continuo, numérico discreto, etc.) Pero en este caso no vamos a realizar ninguna codicación porque no es necesaria.

Nuestro segundo paso en el preprocesado de los datos sería realizar una **selección de características y/o una extracción de características**. Los métodos de **selección** escogen algunas de las variables originales del prolema, estas son más adecuadas cuando no se quiere  comprometer la interpretabilidad de los datos. Sin embargo, si lo que buscamos es eficiencia, los **métodos de extracción** son mejores ya que reducen la dimensionalidad pero preservando la mayor parte de la información de todas las variables originales. En este caso, lo que voy a realizar es para cada caso una cosa, para los datos de clasificación podríamos realizar una selección de características pero como primero vamos a trabajar con los datos de regresión trabajaremos directamente una extracción de características la cual podemos utilizar de cualquier tipo de los vistos en clase (PCA, LLE), en este caso la realización de la PCA no nos sirve porque al tener pocos datos no realiza su función ya que es una técnica que se utiliza para enfatizar la variación y resaltar patrones sólidos en un conjunto de datos. A menudo se utiliza para que los datos sean fáciles de explorar y visualizar.

Igualemte la he realizado para comprobar que lo que digo es cierto, si esto es así y realmente la PCA no es necesaria lo veremos reflejados en los modelos.

Para aplicar la PCA, necesitaremos saber cuantas componenct elegir, por lo tanto con nuestras variables normalizadas y sin normalizar vamos a buscar cual sería la instancia del modelo que vamos a utilizar. Significando PCA (.90), por ejemplo que scikit-learn elige el número mínimo de componentes principales de manera que se retiene el 90% de la varianza. Es lo mismo que utilizamos en las prácticas con explained_variance_ (la cantidad de variación explicada por cada uno de los componentes seleccionados) si no que al revés.

In [ ]:
# Nuestros valores están normalizados, por lo que ahora aplicamos PCA
# Crea una instancia del modelo
pca_100 = PCA(.100)
pca_95 = PCA(.95)
pca_90 = PCA(.90)
pca_85 = PCA(.85)
pca_80 = PCA(.80)

# Ajustamos 
pca_100.fit(X_train_scaled) 
pca_95.fit(X_train_scaled) 
pca_90.fit(X_train_scaled) 
pca_85.fit(X_train_scaled) 
pca_80.fit(X_train_scaled) 

# Imprime el número de componentes generados
print("En este caso, el 100% de la variación equivale a", pca_100.n_components_," componentes principales")
print("En este caso, el 95% de la variación equivale a", pca_95.n_components_," componentes principales")
print("En este caso, el 90% de la variación equivale a", pca_90.n_components_," componentes principales")
print("En este caso, el 85% de la variación equivale a", pca_85.n_components_," componentes principales")
print("En este caso, el 80% de la variación equivale a", pca_80.n_components_," componentes principales")

En este caso, el 100% de la variación equivale a 1  componentes principales
En este caso, el 95% de la variación equivale a 7  componentes principales
En este caso, el 90% de la variación equivale a 6  componentes principales
En este caso, el 85% de la variación equivale a 5  componentes principales
En este caso, el 80% de la variación equivale a 4  componentes principales


In [ ]:
# Eligiendo como lo que creo que sería lo mas correcto como xxx componentes principales:
model_pca = PCA(n_components=7)
model_pca.fit(X_train_scaled) 
Xpca = model_pca.transform(X_train_scaled)
ypca = y_train

# Para ver si se ha hecho bien vamos a mirar los shape de cada uno 
print("Shape Xpca: ", Xpca.shape)
print("Shape ypca: ", ypca.shape)

Shape Xpca:  (309, 7)
Shape ypca:  (309,)


## Desarrollo del modelo 

Una vez realizada el preprocesado de los datos vamos a pasar a la realización de nuestro modelo. Vamos a realizarlo mediante las SVM y los árboles.

El apartado de métodos de partición de datos durante el entrenamiento, he decidido no realizarlo ya que tenemos muy pocos datos y no vale la pena realizar estos pasos, si hubiera sido un data set más grande obviamente hubiera sido mejor 

#### SVR, con PCA

In [ ]:
parameters = {'C': np.logspace(-3,3,10), 'epsilon':np.logspace(-2,2,10)}

clf_svr1 = GridSearchCV(estimator = sklearn.svm.LinearSVR(), param_grid = parameters, n_jobs = -1,cv = 5, verbose = 2, scoring = "neg_mean_absolute_error")
clf_svr1.fit(Xpca, ypca)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.6s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=LinearSVR(C=1.0, dual=True, epsilon=0.0,
                                 fit_intercept=True, intercept_scaling=1.0,
                                 loss='epsilon_insensitive', max_iter=1000,
                                 random_state=None, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                         'epsilon': array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=2)

#### RandomForestRegressor, con PCA

In [ ]:
parameters = {'n_estimators': range(68,128,5), 'max_depth': range(1,8)}

clf_rf1 = GridSearchCV(estimator = RandomForestRegressor(max_features = "sqrt"), param_grid = parameters, n_jobs = -1, cv = 5, verbose = 2, scoring = "neg_mean_absolute_error")
clf_rf1.fit(Xpca, ypca)

Fitting 10 folds for each of 84 candidates, totalling 840 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 682 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done 840 out of 840 | elapsed:   20.9s finished


GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='sqrt',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_j

Pero, ¿Qué pasaría si utilizasemos los datos sin realizar la PCA? Volvamos a realizar el mismo proceso.

#### SVR, sin PCA

In [ ]:
parameters = {'C': np.logspace(-3,3,10), 'epsilon': np.logspace(-2,2,10)}

svr = GridSearchCV(estimator = sklearn.svm.LinearSVR(), param_grid = parameters, n_jobs = -1, cv = 5, verbose = 2, scoring = "neg_mean_absolute_error")
svr.fit(X_train_scaled, ytrain)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.3s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=LinearSVR(C=1.0, dual=True, epsilon=0.0,
                                 fit_intercept=True, intercept_scaling=1.0,
                                 loss='epsilon_insensitive', max_iter=1000,
                                 random_state=None, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                         'epsilon': array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=2)

#### DecisionTreeRegresor sin PCA

In [ ]:
param_grid = [{'max_depth': range(1,10)}]
dtr = GridSearchCV(DecisionTreeRegressor(), param_grid=param_grid, cv=10, verbose=1, n_jobs=-1, scoring = "neg_mean_absolute_error")
dtr.fit(X_train_scaled, ytrain)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    0.1s finished


GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'max_depth': range(1, 10)}], pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False,
             scoring='neg

#### RandomForestRegressor, sin PCA

In [ ]:

rfr = GridSearchCV(RandomForestRegressor(max_features = "sqrt"),
                  param_grid = param_grid,
                  cv = 5,
                  verbose = 1,
                  n_jobs = -1)

rfr.fit(X_train_scaled, ytrain)

Fitting 5 folds for each of 98 candidates, totalling 490 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 490 out of 490 | elapsed:    5.0s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='sqrt',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

#### SVC, sin PCA

In [ ]:
parameters = {'C': np.logspace(-3,3,10)}

svc = GridSearchCV(estimator = sklearn.svm.LinearSVC(), param_grid = parameters, n_jobs = -1, cv = 5, verbose = 2, scoring = "neg_mean_absolute_error")
svc.fit(X_train_scaled, ytrain_class)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


GridSearchCV(cv=5, error_score=nan,
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=2)

#### DecisionTreeClassifier, sin PCA

In [ ]:
param_grid = [{'max_depth': range(1,10)}]
dtc = GridSearchCV(DecisionTreeClassifier(), param_grid=param_grid, cv=10, verbose=1, n_jobs=-1, scoring = "neg_mean_absolute_error")
dtc.fit(X_train_scaled,ytrain_class)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    0.1s finished


GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'max_depth': range(1, 10)}], pre_dispatch='2*n_jobs',

#### RandomForestClassifier, sin PCA

In [ ]:
parameters = {'n_estimators': range(68,100,5), 'max_depth': range(1,10)}

rfc = GridSearchCV(estimator = RandomForestClassifier(max_features = "sqrt"), 
                                 param_grid = parameters, 
                                 n_jobs = -1, cv = 5, 
                                 verbose = 2, 
                                 scoring = "neg_mean_absolute_error")
rfc.fit(X_train_scaled,ytrain_class)

Fitting 5 folds for each of 63 candidates, totalling 315 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 315 out of 315 | elapsed:    7.9s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='sqrt',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

## Validación del modelo 

In [ ]:
print("   ")
print("----------------------------------------------------------------------------")
print("best_score_svr sin PCA: ", svr.best_params_)
print("best_score_dtr sin PCA: ", dtr.best_params_)
print("best_score_rfr sin PCA: ", rfr.best_params_)
print("best_score_svc sin PCA: ", svc.best_params_)
print("best_score_dtc sin PCA: ", dtc.best_params_)
print("best_score_rfc sin PCA: ", rfc.best_params_)
print("----------------------------------------------------------------------------")
print("best_score_svr sin PCA: ", svr.best_score_)
print("best_score_dtr sin PCA: ", dtr.best_score_)
print("best_score_rfr sin PCA: ", rfr.best_score_)
print("best_score_svc sin PCA: ", svc.best_score_)
print("best_score_dtc sin PCA: ", dtc.best_score_)
print("best_score_rfc sin PCA: ", rfc.best_score_)
print("----------------------------------------------------------------------------")

   
----------------------------------------------------------------------------
best_score_svr sin PCA:  {'C': 46.41588833612773, 'epsilon': 0.01}
best_score_dtr sin PCA:  {'max_depth': 3}
best_score_rfr sin PCA:  {'max_depth': 5, 'n_estimators': 55}
best_score_svc sin PCA:  {'C': 46.41588833612773}
best_score_dtc sin PCA:  {'max_depth': 3}
best_score_rfc sin PCA:  {'max_depth': 4, 'n_estimators': 83}
----------------------------------------------------------------------------
best_score_svr sin PCA:  -43.917863517355364
best_score_dtr sin PCA:  -50.1365626481907
best_score_rfr sin PCA:  0.44270328012330956
best_score_svc sin PCA:  -0.45282919090428353
best_score_dtc sin PCA:  -0.4825806451612903
best_score_rfc sin PCA:  -0.41443680592279214
----------------------------------------------------------------------------


## Obtención de conclusiones

In [ ]:
# Inicializamos el predictor para la clasificacion
predictor = DecisionTreeClassifier
criterion = 'entropy'
max_depth = 3
report = sklearn.metrics.classification_report

# predictor
dt = predictor(criterion=criterion, max_depth=max_depth)
dt.fit(Xtrain, ytrain_class)

y_hat = dt.predict(Xtest)
y_hat = np.round(y_hat)

y_hat = pd.DataFrame(y_hat)
y_hat.to_csv('yhat_clasificacion.csv')

In [ ]:
# Inicializamos el predictor para la regresion
predictor = RandomForestRegressor

# predictor
dt = predictor()
dt.fit(Xtrain, ytrain)

y_hat = dt.predict(Xtest)
y_hat = np.round(y_hat)

y_hat = pd.DataFrame(y_hat)
y_hat.to_csv('yhat_regresion.csv')